# Scraping / API :

Le but de ce projet est de créer un algorithme de recommandation de film, pour cela nous devons aller chercher les données nécessaires, c'est à dire une liste de film avec toutes les informations que nous considérons comme utile pour recommander un film. 

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

driver = webdriver.Chrome()

In [2]:
import pandas as pd

J'ai trouvé un top de 10000 films sur le site IMDB, je vais donc m'en servir pour scraper les identifiants de ces films qui se trouvent dans leurs URL respectifs.

In [3]:

# Liste pour stocker les identifiants IMDb
identifiants_imdb = []

# URL de base
base_url = 'https://www.imdb.com/search/title/?num_votes=10000,&sort=user_rating,desc&title_type=feature'

# Boucle pour parcourir les 100 pages (de 1 à 100)
for page_num in range(1, 201):
    # Construire l'URL pour la page actuelle
    page_url = f'{base_url}&start={(page_num - 1) * 50 + 1}'  # Ajustement de la formule ici
    
    # Accéder à l'URL de la page
    driver.get(page_url)
    
    time.sleep(0.5)  # Utiliser la fonction sleep pour laisser le temps à la page de charger 
    
    # Obtenir le contenu de la page
    page_content = driver.page_source
    
    # Extraire les liens IMDb des films
    soup = BeautifulSoup(page_content, 'html.parser')
    imdb_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].startswith('/title/')& a['href'].endswith('i')]

    for href in imdb_links:
        imdb_id = href.split('/')[2]
        identifiants_imdb.append(imdb_id)


Verifier que l'on à bien extrait les 10 000 Films

In [4]:
ensemble_identifiants_imdb = set(identifiants_imdb)
len(ensemble_identifiants_imdb)

10000

Ensuite je vais utiliser l'API TMDB pour obtenir le plus d'information possible sur ces films à partir de leurs identifiants.

In [5]:
import requests
import json
api_key = "***********************"

In [8]:
import requests
import json
from IPython.display import display, clear_output

movie_data = []  # Cette liste stockera les informations sur les films
success_count = 0  # Compteur de requêtes réussies
failure_count = 0  # Compteur de requêtes en échec

for identif_imdb in identifiants_imdb:
    # Créer l'URL en utilisant l'identifiant IMDb actuel
    url = f"https://api.themoviedb.org/3/find/{identif_imdb}?api_key={api_key}&external_source=imdb_id&language=fr"

    time.sleep(1/50)

    # Envoyer une requête GET à l'API TMDb
    response = requests.get(url)
 
    if response.status_code == 200:
        # Si la requête réussit, traiter la réponse JSON
        movie_info = json.loads(response.text)
        movie_data.append(movie_info)
        success_count += 1
    else:
        # Gérer les erreurs en ajoutant un message d'erreur à la liste ou en enregistrant les identifiants IMDb non valides.
        print(f"Erreur pour IMDb ID {identif_imdb}")
        failure_count += 1

    # Afficher le nombre de requêtes effectuées jusqu'à présent
    total_requests = success_count + failure_count
    message = f"Requêtes effectuées : {total_requests}, Réussies : {success_count}, En échec : {failure_count}"
    clear_output(wait=True)
    display(message)

'Requêtes effectuées : 10000, Réussies : 10000, En échec : 0'

Verification du contenu de la liste movie_data

In [9]:
movie_data

[{'movie_results': [{'adult': False,
    'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg',
    'id': 278,
    'title': 'Les Évadés',
    'original_language': 'en',
    'original_title': 'The Shawshank Redemption',
    'overview': "En 1947, Andy Dufresne, un jeune banquier, est condamné à la prison à vie pour le meurtre de sa femme et de son amant. Ayant beau clamer son innocence, il est emprisonné à Shawshank, le pénitencier le plus sévère de l'État du Maine. Il y fait la rencontre de Red, un noir désabusé, détenu depuis vingt ans. Commence alors une grande histoire d'amitié entre les deux hommes…",
    'poster_path': '/xqfTE9FjwD5vS1A6QnyoMdPQUvh.jpg',
    'media_type': 'movie',
    'genre_ids': [18, 80],
    'popularity': 90.97,
    'release_date': '1994-09-23',
    'video': False,
    'vote_average': 8.702,
    'vote_count': 24617}],
  'person_results': [],
  'tv_results': [],
  'tv_episode_results': [],
  'tv_season_results': []},
 {'movie_results': [{'adult': False,
    'backdr

La dernière étape est de stocker les informations dans un dataframe puis dans un fichier CSV

In [25]:
# Créez une liste de dictionnaires pour stocker les informations de chaque film
film_dicts = []
for result in movie_data:
    if 'movie_results' in result:
        movies = result['movie_results']
        for movie in movies:
            film_dicts.append(movie)

# Créez un DataFrame à partir de la liste de dictionnaires
data_movies = pd.DataFrame(film_dicts)
data_movies


,adult,backdrop_path,id,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count
0,False,/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg,278,Les Évadés,en,The Shawshank Redemption,"En 1947, Andy Dufresne, un jeune banquier, est...",/xqfTE9FjwD5vS1A6QnyoMdPQUvh.jpg,movie,"[18, 80]",90.970,1994-09-23,False,8.702,24617
1,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,238,Le Parrain,en,The Godfather,La Seconde Guerre mondiale vient de s'achever....,/wnDNKCeBQzioXYQrXcSyrmRHVxf.jpg,movie,"[18, 80]",95.013,1972-03-14,False,8.706,18651
2,False,/nIgx0Begoi4LmzoI6esEB7TLLbX.jpg,84092,ラーマーヤナ ラーマ王子伝説,hi,ラーマーヤナ ラーマ王子伝説,,/puHViEBI3bG7T8X1VshDQiUMc5H.jpg,movie,"[16, 14, 10751, 28, 12]",5.732,1992-12-05,False,8.200,32
3,False,/qFLQEng8NADXWkxB5vE9dO7YyNf.jpg,83651,Hababam Sınıfı,tr,Hababam Sınıfı,,/rpiU1Y0xNEsxM1bwqFEZZCOqR3J.jpg,movie,"[35, 18]",6.174,1975-04-01,False,7.975,120
4,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,155,The Dark Knight : Le Chevalier noir,en,The Dark Knight,Batman aborde une phase décisive de sa guerre ...,/pyNXnq8QBWoK3b37RS6C3axwUOy.jpg,movie,"[18, 28, 80, 53]",94.684,2008-07-16,False,8.512,30582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,/zogWnCSztU8xvabaepQnAwsOtOt.jpg,531499,The Tax Collector,en,The Tax Collector,David et Creeper travaillent comme « percepteu...,/8DiCiKUi3YGo3VM8cCIkMXsu3ki.jpg,movie,"[28, 80, 18]",25.152,2020-08-07,False,5.647,421
9996,False,/qkBWIZRaeMByMkoby48o71zYyA7.jpg,11811,Leprechaun,en,Leprechaun,Un lutin terrifiant devient fou après qu'on lu...,/mIpzSHNX9lQlCKnh1cQwIXDFX5d.jpg,movie,"[27, 35, 14]",20.284,1993-01-08,False,5.581,687
9997,False,/wa5eEBwjI01yhNMKPwIaqeBiw2d.jpg,9053,DOA: Dead or alive,en,DOA: Dead or Alive,"Quatre femmes, au début rivales dans une compé...",/j6fYGbitK582ZuJy0xiUFZma79q.jpg,movie,"[12, 28, 53]",22.624,2006-09-07,False,5.178,749
9998,False,/oOZM1C3wZtsZtdidVgEazeas7U4.jpg,727745,The Kissing Booth 3,en,The Kissing Booth 3,Dernier été avant qu’Elle Evans n'entre à l’un...,/cM8KCuWsCu02JwEug0ilGJJ7wBU.jpg,movie,"[10749, 35]",26.614,2021-08-11,False,6.981,1722


In [27]:
identifiant = pd.DataFrame(identifiants_imdb)

In [28]:
data_movies['identifiants'] = identifiant

In [29]:
data_movies

,adult,backdrop_path,id,title,original_language,original_title,overview,poster_path,media_type,genre_ids,popularity,release_date,video,vote_average,vote_count,identifiants
0,False,/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg,278,Les Évadés,en,The Shawshank Redemption,"En 1947, Andy Dufresne, un jeune banquier, est...",/xqfTE9FjwD5vS1A6QnyoMdPQUvh.jpg,movie,"[18, 80]",90.970,1994-09-23,False,8.702,24617,tt0111161
1,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,238,Le Parrain,en,The Godfather,La Seconde Guerre mondiale vient de s'achever....,/wnDNKCeBQzioXYQrXcSyrmRHVxf.jpg,movie,"[18, 80]",95.013,1972-03-14,False,8.706,18651,tt0068646
2,False,/nIgx0Begoi4LmzoI6esEB7TLLbX.jpg,84092,ラーマーヤナ ラーマ王子伝説,hi,ラーマーヤナ ラーマ王子伝説,,/puHViEBI3bG7T8X1VshDQiUMc5H.jpg,movie,"[16, 14, 10751, 28, 12]",5.732,1992-12-05,False,8.200,32,tt0259534
3,False,/qFLQEng8NADXWkxB5vE9dO7YyNf.jpg,83651,Hababam Sınıfı,tr,Hababam Sınıfı,,/rpiU1Y0xNEsxM1bwqFEZZCOqR3J.jpg,movie,"[35, 18]",6.174,1975-04-01,False,7.975,120,tt0252487
4,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,155,The Dark Knight : Le Chevalier noir,en,The Dark Knight,Batman aborde une phase décisive de sa guerre ...,/pyNXnq8QBWoK3b37RS6C3axwUOy.jpg,movie,"[18, 28, 80, 53]",94.684,2008-07-16,False,8.512,30582,tt0468569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,/zogWnCSztU8xvabaepQnAwsOtOt.jpg,531499,The Tax Collector,en,The Tax Collector,David et Creeper travaillent comme « percepteu...,/8DiCiKUi3YGo3VM8cCIkMXsu3ki.jpg,movie,"[28, 80, 18]",25.152,2020-08-07,False,5.647,421,tt8461224
9996,False,/qkBWIZRaeMByMkoby48o71zYyA7.jpg,11811,Leprechaun,en,Leprechaun,Un lutin terrifiant devient fou après qu'on lu...,/mIpzSHNX9lQlCKnh1cQwIXDFX5d.jpg,movie,"[27, 35, 14]",20.284,1993-01-08,False,5.581,687,tt0107387
9997,False,/wa5eEBwjI01yhNMKPwIaqeBiw2d.jpg,9053,DOA: Dead or alive,en,DOA: Dead or Alive,"Quatre femmes, au début rivales dans une compé...",/j6fYGbitK582ZuJy0xiUFZma79q.jpg,movie,"[12, 28, 53]",22.624,2006-09-07,False,5.178,749,tt0398913
9998,False,/oOZM1C3wZtsZtdidVgEazeas7U4.jpg,727745,The Kissing Booth 3,en,The Kissing Booth 3,Dernier été avant qu’Elle Evans n'entre à l’un...,/cM8KCuWsCu02JwEug0ilGJJ7wBU.jpg,movie,"[10749, 35]",26.614,2021-08-11,False,6.981,1722,tt12783454


In [30]:
data_movies.to_csv('infos_movie.csv')

Mission accomplie !